In [1]:
import warnings
warnings.filterwarnings('ignore')
#  Ebnable HTML/CSS 
from IPython.core.display import HTML
HTML("<link href='https://fonts.googleapis.com/css?family=Passion+One' rel='stylesheet' type='text/css'><style>div.attn { font-family: 'Helvetica Neue'; font-size: 30px; line-height: 40px; color: #FFFFFF; text-align: center; margin: 30px 0; border-width: 10px 0; border-style: solid; border-color: #5AAAAA; padding: 30px 0; background-color: #DDDDFF; }hr { border: 0; background-color: #ffffff; border-top: 1px solid black; }hr.major { border-top: 10px solid #5AAA5A; }hr.minor { border: none; background-color: #ffffff; border-top: 5px dotted #CC3333; }div.bubble { width: 65%; padding: 20px; background: #DDDDDD; border-radius: 15px; margin: 0 auto; font-style: italic; color: #f00; }em { color: #AAA; }div.c1{visibility:hidden;margin:0;height:0;}div.note{color:red;}</style>")

___
Enter Team Member Names here (double click to edit):

- Name 1:Allen Ansari
- Name 2:Chad Madding
- Name 3:Yongjun Chu

________

# In Class Assignment Three
In the following assignment you will be asked to fill in python code and derivations for a number of different problems. Please read all instructions carefully and turn in the rendered notebook (or HTML of the rendered notebook)  before the end of class.

<a id="top"></a>
## Contents
* <a href="#Loading">Loading the Data</a>
* <a href="#distance">Measuring Distances</a>
* <a href="#KNN">K-Nearest Neighbors</a>
* <a href="#naive">Naive Bayes</a>

________________________________________________________________________________________________________
<a id="Loading"></a>
<a href="#top">Back to Top</a>
## Downloading the Document Data
Please run the following code to read in the "20 newsgroups" dataset from sklearn's data loading module.

In [2]:
from __future__ import print_function
from sklearn.datasets import fetch_20newsgroups_vectorized
import numpy as np


# this takes about 30 seconds to compute, read the next section while this downloads
ds = fetch_20newsgroups_vectorized(subset='train')

# this holds the continuous feature data (which is tfidf)
print('features shape:', ds.data.shape) # there are ~11000 instances and ~130k features per instance
print('target shape:', ds.target.shape) 
print('range of target:', np.min(ds.target),np.max(ds.target))
print('Data type is', type(ds.data), float(ds.data.nnz)/(ds.data.shape[0]*ds.data.shape[1])*100, '% of the data is non-zero')

features shape: (11314, 130107)
target shape: (11314,)
range of target: 0 19
Data type is <class 'scipy.sparse.csr.csr_matrix'> 0.1214353154362896 % of the data is non-zero


## Understanding the Dataset
Look at the description for the 20 newsgroups dataset at http://qwone.com/~jason/20Newsgroups/. You have just downloaded the "vectorized" version of the dataset, which means all the words inside the articles have gone through a transformation that binned them into 130 thousand features related to the words in them.  

**Question Set 1**:
- How many instances are in the dataset? 
- What does each instance represent? 
- How many classes are in the dataset and what does each class represent?
- Would you expect a classifier trained on this data would generalize to documents written in the past week? Why or why not?
- Is the data represented as a sparse or dense matrix?

#### Answers:

There are 11314 instances in the dataset.

Each instance represents an article of a newgroup.

There are 20 classes in the dataset and each class represents one of the twenty newsgroups.

No, we would not expect a classifier trained on this data would generalize to documents written in the past week. This is because the original dataset was created around 1995, and slightly modified at 2008. A lot of new words have emerged during the last 20+ years, like "iPhone", "google", "Netflix". Therefore, the classifier trained on this data would likey mis-classify or have problems classifying documents written in the past week.

The data is represeted as a sparse matrix, since only ~ 0.12% of data is non-zeros.

___
<a id="distance"></a>
<a href="#top">Back to Top</a>
## Measures of Distance
In the following block of code, we isolate three instances from the dataset. The instance "`a`" is from the group *computer graphics*, "`b`" is from from the group *recreation autos*, and "`c`" is from group *recreation motorcycle*. **Exercise for part 2**: Calculate the: 
- (1) Euclidean distance
- (2) Cosine distance 
- (3) Jaccard similarity 

between each pair of instances using the imported functions below. Remember that the Jaccard similarity is only for binary valued vectors, so convert vectors to binary using a threshold. 

**Question for part 2**: Which distance seems more appropriate to use for this data? **Why**?

In [8]:
from scipy.spatial.distance import cosine
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import jaccard
import numpy as np

# get first instance (comp)
idx = 550
a = ds.data[idx].todense()
a_class = ds.target_names[ds.target[idx]]
print('Instance A is from class', a_class)

# get second instance (autos)
idx = 4000
b = ds.data[idx].todense()
b_class = ds.target_names[ds.target[idx]]
print('Instance B is from class', b_class)

# get third instance (motorcycle)
idx = 7000
c = ds.data[idx].todense()
c_class = ds.target_names[ds.target[idx]]
print('Instance C is from class', c_class)

# Enter distance comparison below for each pair of vectors:
print("Euclidean Distance\n ab:", euclidean(a,b), "ac:", euclidean(a,c), "bc:", euclidean(b,c)) 
print("Cosine Distance\n ab:", cosine(a,b), "ac:", cosine(a,c), "bc:", cosine(b,c))
print("Jaccard Dissimilarity (vectors should be boolean values)\n ab:", jaccard(a>0,b>0), "ac:", jaccard(a>0,c>0), "bc:", jaccard(b>0,c>0))

print("\nThe most appropraite distance is Cosine.")
print("Because the distances generated from this method make most sense. \nB and C are closest, and A is about the same distance from B and C.")
print("That is exactly we saw from Cosine distance results.")
print("Euclidean also works well, but not quite as good as “cosine”.")

Instance A is from class comp.graphics
Instance B is from class rec.autos
Instance C is from class rec.motorcycles
Euclidean Distance
 ab: 1.0985184671870858 ac: 1.1891405425398236 bc: 0.9177794226661624
Cosine Distance
 ab: 0.6033714113755322 ac: 0.7070276149559529 bc: 0.4211595343347173
Jaccard Dissimilarity (vectors should be boolean values)
 ab: 0.8821138211382114 ac: 0.8754716981132076 bc: 0.9087947882736156

The most appropraite distance is Cosine.
Because the distances generated from this method make most sense. 
B and C are closest, and A is about the same distance from B and C.
That is exactly we saw from Cosine distance results.
Euclidean also works well, but not quite as good as “cosine”.


___
# Start of Live Session Assignment
___
<a id="KNN"></a>
<a href="#top">Back to Top</a>
## Using scikit-learn with KNN
Now let's use stratified cross validation with a holdout set to train a KNN model in `scikit-learn`. Use the example below to train a KNN classifier. The documentation for `KNeighborsClassifier` is here: http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html  

**Exercise for part 3**: Use the code below to test what value of `n_neighbors` works best for the given data. *Note: do NOT change the metric to be anything other than `'euclidean'`. Other distance functions are not optimized for the amount of data we are working with.* 

**Question for part 3**: What is the accuracy of the best classifier you can create for this data (by changing only the `n_neighbors` parameter)? 

In [10]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from IPython.html import widgets 

sss = StratifiedShuffleSplit(test_size = 0.5, train_size=0.5, random_state=12345)

# fill in the training and testing data and save as separate variables
for trainidx, testidx in sss.split(ds.data,ds.target):
    # note that these are sparse matrices
    X_train = ds.data[trainidx] 
    X_test = ds.data[testidx] 
    y_train = ds.target[trainidx]
    y_test = ds.target[testidx]

# fill in your code  here to train and test
# calculate the accuracy and print it for various values of K
acc_dict = {}
for K in range(1, 11): 
    clf = KNeighborsClassifier(n_neighbors=K, weights='uniform', metric='euclidean')
    clf.fit(X_train, y_train)
    yhat = clf.predict(X_test)
    print('Accuracy of classifier with %d neighbors is: %.6f'%(K,accuracy_score(y_test, yhat)))
    #print(K, accuracy_score(y_test, yhat))
    acc_dict.update( {K : accuracy_score(y_test, yhat)} )
    
    
maximum = max(acc_dict, key=acc_dict.get)  # Just use 'min' instead of 'max' for minimum.
acc = acc_dict[maximum] # overwrite this with the actual accuracy

#=====================================

print('The maximum accuracy of the classifier is %.2f with %d neighbors'%(acc,maximum))

Accuracy of classifier with 1 neighbors is: 0.601556
Accuracy of classifier with 2 neighbors is: 0.558954
Accuracy of classifier with 3 neighbors is: 0.531731
Accuracy of classifier with 4 neighbors is: 0.510871
Accuracy of classifier with 5 neighbors is: 0.491780
Accuracy of classifier with 6 neighbors is: 0.479053
Accuracy of classifier with 7 neighbors is: 0.473926
Accuracy of classifier with 8 neighbors is: 0.467562
Accuracy of classifier with 9 neighbors is: 0.456072
Accuracy of classifier with 10 neighbors is: 0.447941
The maximum accuracy of the classifier is 0.60 with 1 neighbors


**Question for part 3**: With sparse data, does the use of a KDTree representation make sense? Why or Why not?

No. The use of a KDTree representation likely does not make sense with sparse data. The cost of measuring distance with truly sparse data is small--usually smaller than the cost of creating a tree. Sparse data also will not branch well in a KDTree because there are so many zeros. The number of "zero" comparisons will mean each tree needs to be very deep. Therefore, the benefit of the KDTree is drastically reduced.

_____
## KNN extensions - Centroids
Now lets look at a very closely related classifier to KNN, called nearest centroid. In this classifier (which is more appropriate for big data scenarios and sparse data), the training step is used to calculate the centroids for each class. These centroids are saved. Unknown attributes, at prediction time, only need to have distances calculated for each saved centroid, drastically decreasing the time required for a prediction. 

**Exercise for part 4**: Use the template code below to create a nearest centroid classifier. Test which metric has the best cross validated performance: Euclidean, Cosine, or Manhattan. In `scikit-learn` you can see the documentation for NearestCentroid here: 
- http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestCentroid.html#sklearn.neighbors.NearestCentroid

and for supported distance metrics here:
- http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.distance_metrics.html#sklearn.metrics.pairwise.distance_metrics

In [11]:
from sklearn.neighbors.nearest_centroid import NearestCentroid

# the parameters for the nearest centroid metric to test are:
#    l1, l2, and cosine (all are optimized)
for d in ['euclidean', 'cosine', 'manhattan']:
    clf = NearestCentroid(metric=d)
    clf.fit(X_train, y_train)
    yhat = clf.predict(X_test)
    acc = accuracy_score(y_test, yhat)
    #or use: clf.score(X_test, y_test)
    print(d, acc)

print('The best distance metric is: cosine')

euclidean 0.4132932649814389
cosine 0.4811737670143185
manhattan 0.18967650698249955
The best distance metric is: cosine


___
<a id="naive"></a>
<a href="#top">Back to Top</a>
## Naive Bayes Classification
Now let's look at the use of the Naive Bayes classifier. The 20 newsgroups dataset has 20 classes and about 130,000 features per instance. Recall that the Naive Bayes classifer calculates a posterior distribution for each possible class. Each posterior distribution is a multiplication of many conditional distributions: 

$${\arg \max}_{j} \left(p(class=j)\prod_{i} p(attribute=i|class=j) \right)$$

where $p(class=j)$ is the prior and $p(attribute=i|class=j)$ is the conditional probability.

**Question for part 5**: With this many classes and features, how many different conditional probabilities need to be parameterized? How many priors need to be parameterized?

#### Answers: 
Since we have ~130,000 features and 20 classes, we need to calculate 20 * 130,000 or 2.6 million conditional probabilities. There is 1 prior for each class, in a total of 20 priors, that must be parametized.


___
## Naive Bayes in Scikit-learn
Scikit has several implementations of the Naive Bayes classifier: `GaussianNB`, `MultinomialNB`, and `BernoulliNB`. Look at the documentation here: http://scikit-learn.org/stable/modules/naive_bayes.html Take a look at each implementation and then answer this question: 

**Questions for part 6**: 
- If the instances contain mostly continuous attributes, would it be better to use Gaussian Naive Bayes, Multinomial Naive Bayes, or Bernoulli? And Why? 
- What if the data is sparse, does this change your answer? Why or Why not?

#### Answers:

1) For continuous attributes, it would be best to use Guassian Naive Bayes. Gaussian works effectively for continuous datasets as it considers the data through a normalization process to fit a naive Bayes format, although it requires the assumption that the underlying data is normally distributed (which is often approximately true). 


2) If the data is sparse, we will change our answer from Guassian Naive Bayes to Multinomial Naive Bayes. Sparse matrices are much harder to find realistic Gaussian models, because they always have a mean near zero.
- For sparse data, it is probably better (and faster) to use multinomial naive Bayes. 
- An argument can also be made for Bernoulli if binarizing the feature data helps to reduce the complexity of the problem.	 

## Naive Bayes Comparison
For the final section of this notebook let's compare the performance of Naive Bayes for document classification. Look at the parameters for `MultinomialNB`, and `BernoulliNB` (especially `alpha` and `binarize`). 

**Exercise for part 7**: Using the example code below, change the parameters for each classifier and see how accurate you can make the classifiers on the test set. 

**Question for part 7**: Why are these implementations so fast to train? What does the `'alpha'` value control in these models (*i.e.*, how does it change the parameterizations)? 

- Use a for loop to test both classifiers.       
- for clf in [clf_mnb,clf_bnb]:
- clf.fit(X_train, y_train)
- Change the parameter values in the code above to something like 0.001 for alpha and 0.02 for binarize.



In [15]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

a_list = [0.0, 0.001, 0.01, 0.1, 1]
b_list = [0.0, 0.002, 0.02, 0.04, 0.06, 0.08, 0.2]

for a in a_list:
    clf_mnb = MultinomialNB(alpha= a)
    acc = clf_mnb.fit(X_train, y_train).score(X_test, y_test)
    print('Accuracy of MultinomialNB classifier for alpha = %.3f is: %.3f'%(a,acc))
    

for a in a_list:
    print ('='*80)
    for b in b_list:
        clf_bnb = BernoulliNB(alpha=a, binarize=b)
        acc = clf_bnb.fit(X_train, y_train).score(X_test, y_test)
        print("Accuracy of BernoulliNB classifier for (alpha=%.3f, binarize=%.3f) is: %.3f" % (a, b, acc))

Accuracy of MultinomialNB classifier for alpha = 0.000 is: 0.869
Accuracy of MultinomialNB classifier for alpha = 0.001 is: 0.893
Accuracy of MultinomialNB classifier for alpha = 0.010 is: 0.887
Accuracy of MultinomialNB classifier for alpha = 0.100 is: 0.833
Accuracy of MultinomialNB classifier for alpha = 1.000 is: 0.718
Accuracy of BernoulliNB classifier for (alpha=0.000, binarize=0.000) is: 0.856
Accuracy of BernoulliNB classifier for (alpha=0.000, binarize=0.002) is: 0.857
Accuracy of BernoulliNB classifier for (alpha=0.000, binarize=0.020) is: 0.870
Accuracy of BernoulliNB classifier for (alpha=0.000, binarize=0.040) is: 0.847
Accuracy of BernoulliNB classifier for (alpha=0.000, binarize=0.060) is: 0.791
Accuracy of BernoulliNB classifier for (alpha=0.000, binarize=0.080) is: 0.741
Accuracy of BernoulliNB classifier for (alpha=0.000, binarize=0.200) is: 0.349
Accuracy of BernoulliNB classifier for (alpha=0.001, binarize=0.000) is: 0.841
Accuracy of BernoulliNB classifier for (alp

#### Answers:

For MultinomialNB classifier, among 5 different alpha values tested, alpha = 0.001 yielded highest accuracy at 0.893.

For BernoulliNB classifier,with binarize=0.020, it yielded highest accuracy regardless of alpha values chosen.

For BernoulliNB classifier, it appears that when alpha value is less than or equal to 0.1, the accuracy can be pretty high with an appropriate binarize value.

These classifiers are so fast because they allow the decoupling of the class conditional features. This allows comparisons to be made independently which can significantly speed up comparisons in datasets with high dimensionality.

The alpha value controls the smoothing performed on the model, which affects the weight of features that might not have much impact on the classification and prevents zero probabilities in further computations.

________________________________________________________________________________________________________

That's all! Please **upload your rendered notebook to blackboard** and please include **team member names** in the notebook submission.